In [1]:
import time, re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

In [2]:
def initialize_driver(executable_path):

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--incognito")
    # s = Service(executable_path=executable_path)
    # driver = webdriver.Chrome(service=s, options=options)
    # driver.maximize_window()
    # return driver

    # options = ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--incognito")
    # driver = Chrome(executable_path=executable_path, options=options)
    # driver.maximize_window()
    # return driver

    s = Service(executable_path=executable_path)
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    return driver

In [3]:
def scrape_reviews(driver, url):

    driver.get(url)
    time.sleep(3)

    # Click on the reviews button
    reviews_button = driver.find_element(By.LINK_TEXT,'See All Customer Reviews')


    reviews_button.click()
    time.sleep(3)

    total_star_rating = driver.find_element(By.XPATH,'/html/body/div[2]/main/div[1]/div/div/div[1]/div/div/div[3]/div[2]/div/div[1]/div[2]/div/div/div/ul/li[1]/a/div/span[1]').text
    product_name = driver.find_element(By.XPATH,'/html/body/div[2]/main/div[1]/div/div/div[1]/div/div/div[3]/div[2]/div/div[1]/div[1]/div[1]/div/h2/a').text
    total_product_reviews = driver.find_element(By.XPATH,'/html/body/div[2]/main/div[1]/div/div/div[1]/div/div/div[3]/div[2]/div/div[1]/div[2]/div/div/div/ul/li[1]/a/div/span[2]').text

    # Get total number of reviews
    tot_reviews = driver.find_element(By.XPATH, '/html/body/div[2]/main/div[1]/div/div/div[1]/div/div/div[3]/div[2]/div/div[1]/div[2]/div/div/div/ul/li[1]/a/div/span[2]').text
                                                 
    match = re.search(r'\(([\d,]+)\sReviews\)', tot_reviews)
    if match:
        review_count_str = match.group(1)
        # Remove commas from the string and convert to integer
        review_count = int(review_count_str.replace(',', ''))
        #print("Number of reviews:", review_count)
    else:
        print("Number of reviews not found")

    n = round(review_count/20)

    star_rating = []
    review = []
    name = []
    review_title = []
    votes = []

    for j in range(n):
        
        try:

            ul = driver.find_element(By.CLASS_NAME,'reviews-list')
            li_lst = ul.find_elements(By.CLASS_NAME,'review-item')

            for i in li_lst:
                html_content = i.get_attribute('outerHTML')
                review_content = i.find_element(By.CLASS_NAME,'ugc-review-body').text
                reviewer_name = i.find_element(By.CLASS_NAME,'c-overlay-wrapper').text
                
                title_div = i.find_element(By.CLASS_NAME,'review-heading')
                reviewer_star_rating = title_div.find_element(By.CLASS_NAME,'visually-hidden').text[6:7]
                
                soup = BeautifulSoup(html_content,'html.parser')
                each_review_title = soup.find('h4',class_='c-section-title review-title heading-5 v-fw-medium').text

                helpful_votes = soup.find('button',class_='c-button c-button-outline c-button-sm helpfulness-button no-margin-l').text.split('(')[1].split(')')[0]
                    
                star_rating.append(reviewer_star_rating)
                review.append(review_content)
                name.append(reviewer_name)
                review_title.append(each_review_title)
                votes.append(helpful_votes)
                
                # Click the button to navigate to the next page
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '#reviews-accordion > div.page-pagination.mb-300 > div > div:nth-child(2) > ul > li.page.next > a'))
                )
                next_page_button.click()
                time.sleep(3)
        except NoSuchElementException as nsee:
            print(nsee)
            break

        # try:
        #     review_elements = driver.find_elements(By.XPATH, '//*[@id="reviews-accordion"]/div[1]/ul/li')
        #     total_reviews = len(review_elements)
            
        #     for i in range(1, total_reviews + 1):

        #         try:
        #             star = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[2]/div[1]/div/div/p').text[6:7]
        #             rev = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[5]/div/p').text
        #             rev_name = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[1]/div/span/span/button/div').text
        #             title = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[1]/h4').text
        #             helpful_votes = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[7]/div/div/div[1]/button[1]').text.split('(')[1].split(')')[0]

        #             print(star, rev, rev_name, title, helpful_votes)
                    
        #         except Exception as e:
        #             print(e)

        #         star_rating.append(star)
        #         review.append(rev)
        #         name.append(rev_name)
        #         review_title.append(title)
        #         votes.append(helpful_votes)
            
        #     # Click the button to navigate to the next page
        #     next_page_button = WebDriverWait(driver, 10).until(
        #         EC.element_to_be_clickable((By.CSS_SELECTOR, '#reviews-accordion > div.page-pagination.mb-300 > div > div:nth-child(2) > ul > li.page.next > a'))
        #     )
        #     next_page_button.click()
        #     time.sleep(3)
        # except NoSuchElementException as nsee:
        #     print(nsee)
        #     break

    return product_name, total_star_rating, total_product_reviews, star_rating, review, name, review_title, votes

In [4]:
def main():
    urls = [#common
            'https://www.bestbuy.com/site/apple-iphone-14-128gb-unlocked-midnight/6507555.p?skuId=6507555',
            # 'https://www.bestbuy.com/site/samsung-galaxy-s22-ultra-128gb-phantom-black-verizon/6494471.p?skuId=6494471',
            # 'https://www.bestbuy.com/site/hp-deskjet-2755e-wireless-inkjet-printer-with-3-months-of-instant-ink-included-with-hp-white/6454282.p?skuId=6454282',
            # 'https://www.bestbuy.com/site/jbl-flip-5-portable-bluetooth-speaker-black/6356535.p?skuId=6356535',
            # 'https://www.bestbuy.com/site/microsoft-365-family-up-to-6-people-12-month-subscription-activation-required-windows-mac-os-apple-ios-android-digital/6258038.p?skuId=6258038',
            # 'https://www.bestbuy.com/site/turbotax-deluxe-2023-federal-e-file-state-mac-os-windows-digital/6560825.p?skuId=6560825',
            # 'https://www.bestbuy.com/site/hamilton-beach-classic-4-slice-toaster-with-sure-toast-technology-stainless-steel/6518125.p?skuId=6518125',
            # 'https://www.bestbuy.com/site/razor-mx350-dirt-rocket-ebike-w-7-miles-max-operating-range-14-mph-max-speed-small-blue/3698469.p?skuId=3698469',
            # 'https://www.bestbuy.com/site/lg-65-class-c3-series-oled-4k-uhd-smart-webos-tv/6535929.p?skuId=6535929',
            # 'https://www.bestbuy.com/site/ge-1-4-cu-ft-mid-size-microwave-black/3180025.p?skuId=3180025',
            # #BB exclusive
            # 'https://www.bestbuy.com/site/mcafee-total-protection-5-device-antivirus-internet-security-software-1-year-subscription-android-apple-ios-chrome-mac-os-windows-digital/6517326.p?skuId=6517326',
            # 'https://www.bestbuy.com/site/norton-360-deluxe-5-device-antivirus-internet-security-software-vpn-dark-web-monitoring-1-year-subscription-android-mac-os-windows-apple-ios-digital/6346690.p?skuId=6346690',
            # 'https://www.bestbuy.com/site/apple-iphone-13-5g-128gb-unlocked-midnight/6417788.p?skuId=6417788',
            # 'https://www.bestbuy.com/site/samsung-galaxy-s23-ultra-256gb-unlocked-phantom-black/6529723.p?skuId=6529723'
            ]

    driver = initialize_driver('D:\CU Boulder - Data Science\Spring 2024\Data Mining\chromedriver.exe')

    all_review_dfs = []
    all_product_dfs = []

    for url in urls:
        product_name, total_star_rating, total_product_reviews, star_rating, review, name, review_title, votes = scrape_reviews(driver, url)
        review_df = pd.DataFrame({
            'User_Star_rating': star_rating,
            'Review_content': review,
            'Reviewer_name': name,
            'Review_title': review_title,
            'Helpful_votes': votes
        })

        product_df = pd.DataFrame({
        'URL':[url],
        'Product_name':[product_name],
        'Total_star_rating':[total_star_rating],
        'Total_product_reviews':[total_product_reviews]
        })

        all_review_dfs.append(review_df)
        all_product_dfs.append(product_df)

    driver.quit()

    return all_review_dfs, all_product_dfs

In [5]:
# Call the main function to get data for all URLs
all_review_dfs, all_product_dfs = main()

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=122.0.6261.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6054A5E42+3538674]
	(No symbol) [0x00007FF6050C4C02]
	(No symbol) [0x00007FF604F75AEB]
	(No symbol) [0x00007FF604F7AD39]
	(No symbol) [0x00007FF604F7D46A]
	(No symbol) [0x00007FF604FFCA5B]
	(No symbol) [0x00007FF604FDF0AA]
	(No symbol) [0x00007FF604FFBDE3]
	(No symbol) [0x00007FF604FDEE53]
	(No symbol) [0x00007FF604FAF514]
	(No symbol) [0x00007FF604FB0631]
	GetHandleVerifier [0x00007FF6054D6CAD+3738973]
	GetHandleVerifier [0x00007FF60552C506+4089270]
	GetHandleVerifier [0x00007FF605524823+4057299]
	GetHandleVerifier [0x00007FF6051F5C49+720121]
	(No symbol) [0x00007FF6050D126F]
	(No symbol) [0x00007FF6050CC304]
	(No symbol) [0x00007FF6050CC432]
	(No symbol) [0x00007FF

In [ ]:
df_all_reviews_df = pd.concat(all_review_dfs, ignore_index=True)
df_all_reviews_df[18:]

,User_Star_rating,Review_content,Reviewer_name,Review_title,Helpful_votes
18,5,Love it. Easy to use. Great Camera. I brought ...,Grayson21,Grandma loves it,0
19,5,"because I have never won anything in my life, ...",Reviewbyme,Nice app,0
20,5,"because I have never won anything in my life, ...",Reviewbyme,Nice app,0
21,5,"because I have never won anything in my life, ...",Reviewbyme,Nice app,0
22,5,"because I have never won anything in my life, ...",Reviewbyme,Nice app,0
...,...,...,...,...,...
195,4,"because I have never won anything in my life, ...",Reviewbyme,Nice app,0
196,3,"because I have never won anything in my life, ...",Reviewbyme,Nice app,0
197,1,"because I have never won anything in my life, ...",Reviewbyme,Nice app,0
198,5,"because I have never won anything in my life, ...",Reviewbyme,Nice app,0


In [ ]:
df_all_product_df = pd.concat(all_product_dfs, ignore_index=True)
df_all_product_df.head()

,URL,Product_name,Total_star_rating,Total_product_reviews
0,https://www.bestbuy.com/site/apple-iphone-14-1...,Apple - iPhone 14 128GB (Unlocked) - Midnight,4.9,(205 Reviews)


In [ ]:
for i, df in enumerate(all_review_dfs):
    df.to_excel(f'url_{i+1}_review_data.xlsx', index=False)

for i, df in enumerate(all_product_dfs):
    df.to_excel(f'url_{i+1}_product_data.xlsx', index=False)

In [ ]:
def concatenate_excel_files(file_names):
    dfs = []
    for file in file_names:
        df = pd.read_excel(file)
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index=True)
    return concatenated_df


file_names_p = ["url_1_product_data.xlsx", "url_2_product_data.xlsx", "url_3_product_data.xlsx", "url_4_product_data.xlsx", "url_5_product_data.xlsx",
               "url_6_product_data.xlsx", "url_7_product_data.xlsx","url_8_product_data.xlsx","url_9_product_data.xlsx", "url_10_product_data.xlsx",
               "url_11_product_data.xlsx", "url_12_product_data.xlsx", "url_13_product_data.xlsx", "url_14_product_data.xlsx"]

file_names_r = ["url_1_review_data.xlsx", "url_2_review_data.xlsx", "url_3_review_data.xlsx", "url_4_review_data.xlsx", "url_5_review_data.xlsx",
               "url_6_review_data.xlsx", "url_7_review_data.xlsx","url_8_review_data.xlsx","url_9_review_data.xlsx", "url_10_review_data.xlsx",
               "url_11_review_data.xlsx", "url_12_review_data.xlsx", "url_13_review_data.xlsx", "url_14_review_data.xlsx"]
# Concatenate the Excel files
concatenated_df_p = concatenate_excel_files(file_names_p)
concatenated_df_r = concatenate_excel_files(file_names_r)

# Write the concatenated DataFrame to a new Excel file
concatenated_df_p.to_excel("Product_data_all.xlsx", index=False)
concatenated_df_r.to_excel("Review_data_all.xlsx", index=False)

print("Concatenation complete. New Excel file created.")

Concatenation complete. New Excel file created.
